In [11]:
import wandb
wandb.login()

import pandas as pd

import params

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [12]:
wandb.init(project=params.PROJECT, entity=params.ENTITY, job_type="baseline_model")

In [13]:
def get_df_wandb():
    data_artifact = wandb.use_artifact(f"{params.DATA_ART}:latest")

    crop_table = data_artifact.get("crop_data_table")
    fertilizer_table = data_artifact.get("fertilizer_data_table")

    df_crop = pd.DataFrame(data=crop_table.data, columns=crop_table.columns)
    df_fertilizer = pd.DataFrame(data=fertilizer_table.data, columns=fertilizer_table.columns)

    return df_crop, df_fertilizer

df_crop, df_fertilizer = get_df_wandb()

wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
